In [8]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

In [9]:
data = pd.read_feather("train.f")

# Repeated Holdout 60/10

In [13]:
results_val = []

for seed in range(20):

    np.random.seed(seed)
    split_point = np.random.randint(72,108)
    print("Repetition {} - Split point {}".format(seed, split_point))
    

    train = data[(data['era'] > split_point - 72) & (data['era'] <= split_point)]
    val = data[(data['era'] > split_point) & (data['era'] <= split_point+12)]

    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Repetition 0 - Split point 72
Correlation 0.055674761932890705

Repetition 1 - Split point 84
Correlation 0.050992191837140016

Repetition 2 - Split point 87
Correlation 0.05715271692259199

Repetition 3 - Split point 96
Correlation 0.04149089173048967

Repetition 4 - Split point 77
Correlation 0.04135711327326403

Repetition 5 - Split point 107
Correlation 0.04742937540341001

Repetition 6 - Split point 82
Correlation 0.04420278564499744

Repetition 7 - Split point 76
Correlation 0.05223502735528707

Repetition 8 - Split point 75
Correlation 0.053688922488772946

Repetition 9 - Split point 100
Correlation 0.03577485782903309

Repetition 10 - Split point 81
Correlation 0.04490352396008006

Repetition 11 - Split point 97
Correlation 0.0439647058970206

Repetition 12 - Split point 83
Correlation 0.043575720545582626

Repetition 13 - Split point 90
Correlation 0.05446284253845101

Repetition 14 - Split point 96
Correlation 0.04149089173048967

Repetition 15 - Split point 80
Correlation 0.

In [26]:
np.median(results_val)

0.04921078362027501

In [27]:
np.min(results_val)

0.03577485782903309

In [28]:
np.max(results_val)

0.05715271692259199

In [22]:
np.mean(results_val)

0.04709690981025643

In [18]:
len(results_val)

20